In [2]:
import pandas 
import re 
import pdfplumber
import os   

In [8]:
file="purchase bil.PDF"
data=[]
with pdfplumber.open(file) as pdf:
    pages=pdf.pages
    text = ""
    for page in pages:
        text += page.extract_text(layout=True)
    text = text.splitlines()
for line in text:
    line=line.strip()
    if re.match(r"^\d{2}/\d{2}/\d{2}",line):
        one=line.split(' ')
        fin=[one[0],one[1],one[-2],one[-1]]
        [one.remove(i) for i in fin] 
        name=' '.join(one).strip()
        fin.append(name)
        data.append(fin)
df=pandas.DataFrame(data,columns=['date','bill no.','GST No.','amount',"supplierName"])
os.makedirs('cleanMAN',exist_ok=True)
firl_path=os.path.join("cleanMan","pur bill.csv")
df.to_csv(firl_path,index=False)

In [3]:
def pdf_col_multi(folder):
    folder_path = folder  # Replace with your folder path
    files = os.listdir(folder_path)
    filedict={}
    for file in files:
        filepath=os.path.join(folder_path,file)
        with pdfplumber.open(filepath) as pdf:
            pages=pdf.pages
            text = ""
            for page in pages:
                text += page.extract_text(layout=True)
            filedict[file] = text.splitlines()
    return filedict
def cleanerMulti(dictu):
    data={}
    for key,value in dictu.items():
        for line in value:
            line=line.strip()
            if re.match(r"^\d{2}/\d{2}/\d{2}",line):
                if key in data.keys():
                    data[key].append(line.split())
                else:
                    data[key]=[line.split()]
    return data
def wordjoiner(data):
    ele=[]
    stru=' '.join([i for i in data if not i.replace(",","").replace(".","").isdigit()])
    ele.append(stru)
    ele += [i for i in data if i.replace(",","").replace(".","").isdigit()]
    return ele
def folderMAN(folderName,dictData,header):
    os.makedirs('cleanMAN',exist_ok=True)
    folder_path = os.path.join('cleanMAN', folderName)
    os.makedirs(folder_path,exist_ok=True)
    for filename,data in dictData.items():
        df=pandas.DataFrame(data,columns=header)
        filename=re.sub(r'\..*$','.csv',filename)
        firl_path=os.path.join(folder_path,filename)
        df.to_csv(firl_path,index=False)
    print('saved')


In [19]:
dictu=pdf_col_multi('daywise sales')
def cleanerMultidws(dictu):
    data={}
    for key,value in dictu.items():
        for line in value:
            line=line.strip()
            if re.match(r"^\d{2}/\d{2}/\d{2}",line):
                arrai=[]
                splat=line.split(' ')
                arrai.append(splat[0])
                arrai.append(re.search(r'\((.*?)\)', line).group(1).strip())
                bills=line.split('(')[0].split(' ')[1:]
                arrai.append(''.join(bills))
                last3=line.split(')')[1].split(' ')
                last3=[x for x in last3 if x]
                if len(last3)==4:
                    arrai+=last3
                else:
                    arrai+=['']+last3
                if key in data.keys():
                    data[key].append(arrai)
                else:
                    data[key]=[arrai]
    return data
clean=cleanerMultidws(dictu)
folderMAN('daywise sales',clean,['BILL DATE','no. of BILL NUMBER',"BILL NUMBERS",'DISC.AMT','CASH SALES','CR.SALES','TOTAL SALES'])


saved


In [35]:
dictu=pdf_col_multi('consol clossing stock 2')
def cleanerMulticon(dictu):
    data={}
    for key,value in dictu.items():
        for line in value:
            line=line.strip()
            if not line.startswith(('SHREE CHENDUR','COMPANY','DOS','-',"T","*")) and line:
                line=line.split()
                ele=[]
                stru=' '.join([i for i in line if not i.replace(",","").replace(".","").isdigit()])
                ele.append(stru)
                ele += [i for i in line if i.replace(",","").replace(".","").isdigit()]
                if key in data.keys():
                    data[key].append(ele)
                else:
                    data[key]=[ele]
    return data
clean=cleanerMulticon(dictu)
folderMAN('consol clossing stock 2',clean,['COMPANY NAME','OPENING','PURCHASE','SALES','CLOSING'])


saved


In [87]:
dictuu=pdf_col_multi('consol closing vat')
def cleanerMulticon(dictu):
    data={}
    for key,value in dictu.items():
        for line in value:
            line=line.strip()
            if re.match(r'^\d+',line) and line:
                line=line.split()
                if line[0].isdigit():
                    finlist=wordjoiner(line[1:])
                    if len(finlist)!=5:
                        finlist.append('')
                    if key in data.keys():
                        data[key].append(finlist)
                    else:
                        data[key]=[finlist]
    return data
clean=cleanerMulticon(dictuu)
#

In [5]:
clean

{'apr23.PDF': [['DELETUS D', '9', '90.310', '90.31', ''],
  ['DUOGEN CYPO', '21', '12.040', '252.84', ''],
  ['DUOGEN DESO', '42', '11.160', '468.72', ''],
  ['DUOGEN DROS', '21', '19.560', '410.76', ''],
  ['FIT FORTE TAB', '10', '8.600', '86.00', ''],
  ['OVIPAUZ-L TAB', '8', '64.380', '515.04', ''],
  ['spasmowin tab', '50', '3.879', '193.95', ''],
  ['ZENFLOX PLUS TAB', '84', '11.421', '45.68', ''],
  ['ACEACT-SP TAB', '2', '8.250', '14.73', '1.77'],
  ['ACEMAXX-P tab', '220', '3.819', '750.20', '90.02'],
  ['ACEMAXX-SP tab', '21', '7.638', '143.22', '17.19'],
  ['ACEMAXX-TH Tab', '206', '12.992', '2389.60', '286.75'],
  ['ACIBAN 20MG', '3', '2.968', '7.95', '0.95'],
  ['ALPRAX 0.25MG TAB', '54', '1.701', '82.03', '9.84'],
  ['ALPRAX 0.5MG TAB', '45', '3.568', '143.37', '17.20'],
  ['Amlogard 5mg', '100', '2.153', '222.65', '26.72'],
  ['AMLONG', '10', '60', '5.056', '270.84', '32.50', ''],
  ['AMLONG 5MG TAB', '45', '2.589', '100.67', '12.08'],
  ['ANDIAL TAB', '2', '1.848', '3.30

In [94]:

folderMAN('consol clossing vat',clean,['Product','QTY','UNIT RATE','STOCK VALUE', 'TAX AMOUNT'])

saved
